In [27]:
#import modules
import pandas as pd

#list all csv files only
csv_files = ['Data/tns_search (14).csv', 'Data/tns_search (5).csv', 'Data/tns_search (6).csv', 'Data/tns_search (7).csv', 'Data/tns_search (8).csv', 'Data/tns_search (9).csv']


In [28]:
df_append = pd.DataFrame()

#append all files together
for file in csv_files:
    df_temp = pd.read_csv(file)
    df_append = pd.concat([df_append, df_temp], ignore_index=True)


In [29]:
df_append.to_csv("Data/ZTF_info.csv")

In [34]:
info = pd.read_csv("Data/ZTF_info.csv")
downloading_info = pd.concat([info["Disc. Internal Name"], info["Discovery Date (UT)"], info["RA"], info["DEC"]], axis = 1, ignore_index = True)


In [41]:
import numpy as np
np.savetxt("ZTF_api/ZTF_info.txt", downloading_info.values, fmt = "%s")

In [66]:
from astropy.time import Time

times = Time(list(info["Discovery Date (UT)"]), format = "iso", scale='utc')
times_JD = times.jd
times_JD_start = times_JD - 100
times_JD_end = times_JD + 500

In [47]:
from astropy.coordinates import SkyCoord
from astropy import units as u
ra_dec = SkyCoord(info["RA"].values, info["DEC"].values, unit=(u.hourangle, u.deg))
ra_degree = ra_dec.ra.degree
dec_degree = ra_dec.dec.degree

In [67]:
downloading_info = np.column_stack((ra_degree[1:3], dec_degree[1:3], times_JD_start[1:3], times_JD_end[1:3]))
np.savetxt("ZTF_api/ZTF_info.txt", downloading_info, fmt = "%s")